# Art Gallery Problem

### Imports

In [43]:
from functools import reduce
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Geometrical Helper Classes

In [4]:
# Point: (x|y)
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f"{self.__class__.__name__}(x={self.x},y={self.y})"
    def __str__(self):
        return f"({self.x}|{self.y})"

In [88]:
# Line with equation: ax + by = c
class Line:
    def __init__(self,a,b,c):
        self.a = a
        self.b = b
        self.c = c
    def __contains__(self, point):
        return point.x * self.a + point.y * self.b == self.c
    def __repr__(self):
        return f"{self.__class__.__name__}(a={self.a},b={self.b},c={self.c})"
    def __str__(self):
        return f"{self.a}x + {self.b}y = {self.c}"
    # Intersections
    def __and__(self, other):
        if self.a*other.b-other.a*self.b == 0:
            return None
        return Point((other.b*self.c-self.b*other.c)/(self.a*other.b-other.a*self.b),\
                     (self.a*other.c - other.a*self.c)/(self.a*other.b-other.a*self.b))

### Line Segment
Next, we take a look at line segments. The class LineSegment will be subclass of Line, but instead of the paramters $a$, $b$ and $c$ it will take the starting and endpoint of the line segment as parameters.

#### How to calculate the paramaters?
We want to calculate the equation $ax+by=c$ of the line passing through our line segment. Let us consider a line segment with endpoints $(x_0|y_0)$ and $(x_1|y_1)$. Assuming at first that $x_0\neq x_1$, we remember the equations for linear functions $y=mx+n$ with
$$m=\frac{y_1-y_0}{x_1-x_0}$$
and 
$$n=y_1-x_1\cdot\frac{y_1-y_0}{x_1-x_0}.$$
By setting $a=m$, $b=-1$ and $c=-n$, we get
$$\frac{y_1-y_0}{x_1-x_0}\cdot x - y = -y_1+x_1\cdot\frac{y_1-y_0}{x_1-x_0}.$$
Multiplying with $(x_1-x_0)$ yields
$$(y_1-y_0)x + (x_0-x_1)y = (x_0-x_1)y_1 + (y_1-y_0)x_1,$$
which already looks quite symmetrical. Indeed, this equation also holds for $x_0=x_1$, giving $x=x_1$. So in general, we have
$$a=y_1-y_0,\\
b=x_0-x_1,\\
c=(x_0-x_1)y_1+(y_1-y_0)x_1.$$

In [95]:
# Line segment from Point point0 to Point point1
class LineSegment(Line):
    def __init__(self,point0,point1):
        self.point0 = point0
        self.point1 = point1
        
        self.a = point1.y - point0.y
        self.b = point0.x - point1.x
        self.c = (point0.x - point1.x)*point1.y + (point1.y - point0.y)*point1.x
    def __contains__(self, point):
        if not super().__contains__(point):
            return False
        return (self.point0.x <= point.x <= self.point1.x or self.point0.x >= point.x >= self.point1.x) \
                and (self.point0.y <= point.y <= self.point1.y or self.point0.y >= point.y >= self.point1.y)
    def fig(self, fig=figure(width=500, height=500)):
        fig.line([self.point0.x, self.point1.x], [self.point0.y, self.point1.y],\
                 color="firebrick", line_width=1)
        return fig
    def __repr__(self):
        show(self.fig())
        return f"{self.__class__.__name__}({repr(self.point0)},{repr(self.point1)})"
    def __str__(self):
        return f"{self.point0} <-> {self.point1}"
    def __and__(self, other):
        intersect = super().__and__(other)
        if intersect in self:
            return intersect
        else:
            return None

In [83]:
# Polygon through the Array of Points points
class Polygon:
    def __init__(self, *points):
        self.points = points
        
        self.segments = [LineSegment(points[i], points[(i+1)%len(points)]) for i in range(len(points))]
    def fig(self, fig=None):
        if not fig:
            fig = figure(width=500, height=500)
        add_figs = lambda fig, obj: obj.fig(fig)
        return reduce(add_figs, [fig] + self.segments)
    def __repr__(self):
        show(self.fig())
        return f"{self.__class__.__name__}({self.points})"

In [20]:
gon = Polygon(Point(2,3),Point(2,3),Point(3,4))

In [97]:
LineSegment(Point(1,1),Point(5,5)) & LineSegment(Point(1,5), Point(5,1))

Point(x=3.0,y=3.0)

In [85]:
Polygon(Point(2,7),Point(7,8), Point(1,1), Point(3,4), Point(5,8), Point(3,3))

Polygon((Point(x=2,y=7), Point(x=7,y=8), Point(x=1,y=1), Point(x=3,y=4), Point(x=5,y=8), Point(x=3,y=3)))

In [80]:
Polygon(Point(7,8), Point(1,1), Point(3,4), Point(5,8), Point(35,7))

Polygon((Point(x=7,y=8), Point(x=1,y=1), Point(x=3,y=4), Point(x=5,y=8), Point(x=35,y=7)))

In [77]:
Polygon(Point(1,1), Point(2,1), Point(2,2), Point(1.5,2.5), Point(1,2))

Polygon((Point(x=1,y=1), Point(x=2,y=1), Point(x=2,y=2), Point(x=1.5,y=2.5), Point(x=1,y=2)))